In [35]:
#https://chronicdata.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-Place-Data-202/eav7-hnsx
import pandas as pd
#df_medical = pd.read_csv('https://chronicdata.cdc.gov/api/views/eav7-hnsx/rows.csv?accessType=DOWNLOAD&bom=true&format=true') # Takes a long time to load
df_medical = pd.read_csv('PLACES__Local_Data_for_Better_Health__Place_Data_2020_release.csv')

In [36]:
df_cancerData = df_medical[(df_medical['MeasureId']=='CANCER') & (df_medical['Data_Value_Type']=='Age-adjusted prevalence')]
df_asthmaData = df_medical[(df_medical['MeasureId']=='CANCER') & (df_medical['Data_Value_Type']=='Age-adjusted prevalence')]
del df_medical # free up any space

In [37]:
df_asthmaData_important = df_asthmaData[['StateAbbr','LocationName','Data_Value','Low_Confidence_Limit','High_Confidence_Limit']] 
df_cancerData_important = df_cancerData[['StateAbbr','LocationName','Data_Value','Low_Confidence_Limit','High_Confidence_Limit']] 

In [38]:
df_TRI_2018 = pd.read_csv('https://enviro.epa.gov/enviro/efservice/MV_TRI_BASIC_DOWNLOAD/year/=/2018/fname/TRI_2019_US.csv/CSV',low_memory=False)

In [39]:
df_TRI = df_TRI_2018[['6. CITY','8. ST','20. INDUSTRY SECTOR','34. CHEMICAL','42. CARCINOGEN','45. 5.1 - FUGITIVE AIR','46. 5.2 - STACK AIR','47. 5.3 - WATER','101. TOTAL RELEASES']]

In [40]:
df_TRI

,6. CITY,8. ST,20. INDUSTRY SECTOR,34. CHEMICAL,42. CARCINOGEN,45. 5.1 - FUGITIVE AIR,46. 5.2 - STACK AIR,47. 5.3 - WATER,101. TOTAL RELEASES
0,NEENAH,WI,Food,Nitrate compounds (water dissociable; reportab...,NO,0.0,0.0,0.00,0.000
1,BRYAN,TX,Nonmetallic Mineral Product,Formic acid,NO,0.0,2692.0,0.00,2692.000
2,CHICAGO,IL,Fabricated Metals,Xylene (mixed isomers),NO,219.0,1878.0,0.00,2097.000
3,HOUSTON,TX,Chemicals,Ammonia,NO,873.8,3154.0,887.46,4923.268
4,HOUSTON,TX,Chemicals,Chlorothalonil,YES,395.2,3393.4,9.02,145433.817
...,...,...,...,...,...,...,...,...,...
80358,PARIS,KY,Chemicals,Methyl isobutyl ketone,YES,0.0,0.0,0.00,0.000
80359,PARIS,KY,Chemicals,Lead compounds,NO,0.0,0.0,0.00,0.000
80360,TURNERS FALLS,MA,Fabricated Metals,Nickel,YES,0.0,0.0,0.00,0.000
80361,GALLATIN,TN,Electric Utilities,Zinc compounds,NO,3.0,970.0,270.00,35243.000


Now we have the base datasets inmplemented, need to learn how to merge them

Lets start withe the carcinogens cause thats the most straightforward:

In [45]:
df_carcinogen = df_TRI[df_TRI['42. CARCINOGEN']=='YES']
# Now lets group by city and state:
gb_carcinogen_by_loc = df_carcinogen.groupby(['8. ST','6. CITY'])["101. TOTAL RELEASES"].sum()

In [46]:
print(gb_carcinogen_by_loc)

8. ST  6. CITY       
AK     ANCHORAGE          1097.74000
       BETHEL              441.98000
       DELTA JUNCTION    36943.40000
       DUTCH HARBOR       1637.70000
       EIELSON AFB           0.13300
                            ...     
WY     ROCK SPRINGS      28523.24200
       SHERIDAN              2.66000
       SINCLAIR           8493.78750
       WHEATLAND         53239.72769
       WORLAND               1.00000
Name: 101. TOTAL RELEASES, Length: 5004, dtype: float64


In [53]:
df_carcinogen_by_loc = gb_carcinogen_by_loc.reset_index()
df_carcinogen_by_loc

,index,8. ST,6. CITY,101. TOTAL RELEASES
0,0,AK,ANCHORAGE,1097.74000
1,1,AK,BETHEL,441.98000
2,2,AK,DELTA JUNCTION,36943.40000
3,3,AK,DUTCH HARBOR,1637.70000
4,4,AK,EIELSON AFB,0.13300
...,...,...,...,...
4999,4999,WY,ROCK SPRINGS,28523.24200
5000,5000,WY,SHERIDAN,2.66000
5001,5001,WY,SINCLAIR,8493.78750
5002,5002,WY,WHEATLAND,53239.72769


In [44]:
df_cancerData_important

,StateAbbr,LocationName,Data_Value,Low_Confidence_Limit,High_Confidence_Limit
1234,AL,Coaling,6.4,6.2,6.7
1301,AL,Cottonwood,6.6,6.3,6.9
1357,AL,Fayette,6.6,6.4,6.8
1375,AL,Guin,6.7,6.4,7.0
1389,AL,Dunnavant,6.7,6.4,7.0
...,...,...,...,...,...
1593099,WI,Zoar,5.3,5.0,5.7
1593106,WI,Westboro,6.5,6.1,6.8
1593114,WY,Guernsey,6.3,6.0,6.6
1593177,WI,Steuben,6.5,6.2,6.9


In [ ]:
# We have way more PLACES data than TRI... 5004 rows versus 28,484

In [56]:
df_cancerData_important[df_cancerData_important['LocationName']=='Bethel']

,StateAbbr,LocationName,Data_Value,Low_Confidence_Limit,High_Confidence_Limit
33362,AK,Bethel,6.5,6.3,6.8
214493,CT,Bethel,6.6,6.4,6.8
215515,DE,Bethel,6.7,6.4,7.0
680012,MN,Bethel,6.3,6.0,6.7
742164,MO,Bethel,6.6,6.2,6.9
981601,NC,Bethel,6.2,5.9,6.4
1041641,OH,Bethel,6.5,6.2,6.9
1183259,PA,Bethel,6.4,6.1,6.7
1446463,VT,Bethel,6.6,6.3,7.0
1489868,WA,Bethel,6.6,6.3,6.8


In [55]:
df_carcinogen_by_loc[df_carcinogen_by_loc['6. CITY']=='BETHEL']

,index,8. ST,6. CITY,101. TOTAL RELEASES
1,1,AK,BETHEL,441.980
533,533,CT,BETHEL,39.136


Looks like to join ill have to correct the location names to be lowercase for both to standardize:

In [62]:
df_carcinogen_by_loc['6. CITY'] = df_carcinogen_by_loc['6. CITY'].str.lower()

In [63]:
df_cancerData_important['LocationName']= df_cancerData_important['LocationName'].str.lower()

c:\users\matth\desktop\cisc367\cisc367_dir\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [64]:
df_cancerData_important['LocationName'].str.lower()

1234          coaling
1301       cottonwood
1357          fayette
1375             guin
1389        dunnavant
              ...    
1593099          zoar
1593106      westboro
1593114      guernsey
1593177       steuben
1593202       bairoil
Name: LocationName, Length: 28484, dtype: object

In [74]:
combined_cancer = pd.merge(df_cancerData_important, df_carcinogen_by_loc, left_on=['StateAbbr','LocationName'],right_on=['8. ST','6. CITY'], how='right')

In [86]:
combined_cancer = combined_cancer.dropna()
combined_cancer = combined_cancer.drop(columns=['index','8. ST','6. CITY'],index=1)

In [87]:
newLabels = ["State","City","Age-adjusted cancer rate (%)", "Lower CI for cancer rate","Upper CI for cancer rate","Total carcinegon release (lb)"]

In [88]:
combined_cancer.columns = newLabels
combined_cancer

,State,City,Age-adjusted cancer rate (%),Lower CI for cancer rate,Upper CI for cancer rate,Total carcinegon release (lb)
0,AK,anchorage,6.3,6.2,6.3,1097.74000
2,AK,delta junction,6.5,6.2,6.8,36943.40000
4,AK,eielson afb,4.7,4.4,5.0,0.13300
5,AK,fairbanks,6.2,6.1,6.3,183054.10890
6,AK,fort greely,5.5,5.2,5.8,0.00000
...,...,...,...,...,...,...
5014,WY,rock springs,6.5,6.3,6.6,28523.24200
5015,WY,sheridan,6.4,6.3,6.6,2.66000
5016,WY,sinclair,6.4,6.1,6.7,8493.78750
5017,WY,wheatland,6.5,6.2,6.8,53239.72769


In [ ]:
# may want to join by the left and add columns without carcinegon release to the "control" dataset